In [1]:
CREATE SCHEMA Bank; 

Commands completed successfully.

Total execution time: 00:00:00.066

In [2]:
CREATE TABLE [Bank].[AccountDetails]
(
  id INT PRIMARY KEY IDENTITY(1, 1), 
  acc_number VARCHAR(40), 
  acc_holder_name VARCHAR(100), 
  balance int
);

Commands completed successfully.

Total execution time: 00:00:00.051

In [3]:
INSERT INTO [Bank].[AccountDetails]
VALUES
('7Z290387', 'Ken', '90');

(1 row affected)

Total execution time: 00:00:00.045

In [2]:
SELECT * FROM [Bank].[AccountDetails]

(1 row affected)

Total execution time: 00:00:00.041

id acc_number acc_holder_name balance 1 7Z290387 Ken 90

**Transaction Isolation Levels**

1 = READ UNCOMMITTED

2 = READ COMMITTED

3 = REPEATABLE READ

4 = SERIALIZABLE

5 = SNAPSHOT

In [5]:
SELECT @@SPID AS server_process_id, transaction_isolation_level
FROM [sys].[dm_exec_requests]
WHERE session_id = @@SPID

(1 row affected)

Total execution time: 00:00:00.289

client_id transaction_isolation_level 55 2

In [4]:
BEGIN TRAN;
  UPDATE [Bank].[AccountDetails]
  SET balance = balance - 45
  WHERE acc_number = '7Z290387';

  WAITFOR DELAY '00:03:00';
ROLLBACK TRAN;


(1 row affected)

Total execution time: 00:03:00.171

In [6]:
SELECT *
FROM [Bank].[AccountDetails]
WHERE acc_number = '7Z290387';

(1 row affected)

Total execution time: 00:00:00.358

id acc_number acc_holder_name balance 1 7Z290387 Ken 90

(1 row affected)

Total execution time: 00:00:00.093

## From this point, we focus on the **Repeatable Read** transaction isolation level

In [7]:
INSERT INTO [Bank].[AccountDetails]
VALUES
('9Z199199', 'Tonya', '200');

(1 row affected)

Total execution time: 00:00:00.409

In [8]:
SELECT * FROM [Bank].[AccountDetails]

(2 rows affected)

Total execution time: 00:00:00.029

id acc_number acc_holder_name balance 1 7Z290387 Ken 90 2 9Z199199 Tonya 200

In [10]:
DECLARE @KenBalance int

BEGIN TRAN;

  SELECT @KenBalance = balance FROM [Bank].[AccountDetails] 
                               WHERE acc_holder_name = 'Ken';
  PRINT 'KenBalance before the delay is: ' + CAST(@KenBalance AS VARCHAR(16));

  WAITFOR DELAY '00:03:00';

  SELECT @KenBalance = balance FROM [Bank].[AccountDetails] 
                               WHERE acc_holder_name = 'Ken';
  PRINT 'KenBalance after the delay is: ' + CAST(@KenBalance AS VARCHAR(16));

ROLLBACK TRAN;

KenBalance before the delay is: 90

KenBalance after the delay is: 45

Total execution time: 00:03:00.204

In [12]:
SET TRANSACTION ISOLATION LEVEL REPEATABLE READ;

DECLARE @KenBalance int;

BEGIN TRAN;

  SELECT @KenBalance = balance FROM [Bank].[AccountDetails] 
                               WHERE acc_holder_name = 'Ken';
  PRINT 'KenBalance before the delay is: ' + CAST(@KenBalance AS VARCHAR(16));

  WAITFOR DELAY '00:03:00';

  SELECT @KenBalance = balance FROM [Bank].[AccountDetails] 
                               WHERE acc_holder_name = 'Ken';
  PRINT 'KenBalance after the delay is: ' + CAST(@KenBalance AS VARCHAR(16));

ROLLBACK TRAN;

KenBalance before the delay is: 45

KenBalance after the delay is: 45

Total execution time: 00:03:00.287